# WORD EMBEDDING

In [ ]:
#pip install tensorflow

In [ ]:
# pip install gensim

In [48]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [49]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import re

In [50]:
df = pd.read_csv(r"C:/Users/kevin/0_projet_satisfaction_client/10_Streamlit/reviews_trust.csv")
df = df[["Commentaire", "star"]]
df["Sentiment"] = df["star"].apply(lambda x : np.where(x >=4 , 1 , 0))  # ajout de la colonne Sentiment à df


In [51]:
#Suppression des valeurs manquantes de la colonne Commentaire
df = df.dropna(axis = 0, how = 'any', subset =["Commentaire"])
df.reset_index(inplace = True)
df = df.drop(['index','star'],axis = 1)

In [52]:
df["preprocessed"] = df["Commentaire"]

#séparation des données pour l'analyse des sentiments
X = df.drop(["Commentaire", "Sentiment"] , axis=1)
y = df["Sentiment"]


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state = 0)

In [53]:
! pip install gensim

In [54]:
import gensim

In [55]:
from preprocessed_class_word2vec import comment_cleaner
from sklearn.linear_model import LogisticRegression
from pipeline_w2v import GensimWord2VecVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline

In [56]:
# Best score training
from joblib import dump, load
import time

####
#tfidf - LOGREG
start = time.time()
clf_lr_w2v = LogisticRegression(max_iter= 10000, C = 10, penalty = 'l2', solver = 'liblinear')
end = time.time()
print("Durée d'entraînement :", end - start, "secondes")

scaler = MaxAbsScaler()


Durée d'entraînement : 0.0 secondes


## Sentiment analysis

### word2vec 

### Logistic regression

In [57]:
####
# pipe W2v - regression logistique
pipe_w2v_lr = Pipeline(steps=[
    ('clean_comment',comment_cleaner()),
    ('vectorizer', GensimWord2VecVectorizer()),
    ('scaler', scaler),
    ('classifieur',clf_lr_w2v)], 
    verbose= True)

In [58]:
X_train = np.array(X_train)
pipe_w2v_lr.fit(X_train.reshape(-1,1),y_train)

[Pipeline] ..... (step 1 of 4) Processing clean_comment, total=   8.5s
[Pipeline] ........ (step 2 of 4) Processing vectorizer, total=   3.9s
[Pipeline] ............ (step 3 of 4) Processing scaler, total=   0.0s
[Pipeline] ....... (step 4 of 4) Processing classifieur, total=   0.8s


Pipeline(memory=None,
         steps=[('clean_comment', comment_cleaner()),
                ('vectorizer',
                 GensimWord2VecVectorizer(hs=0, min_count=2, negative=10,
                                          seed=34, sg=1, vector_size=200,
                                          window=5, workers=-1)),
                ('scaler', MaxAbsScaler(copy=True)),
                ('classifieur',
                 LogisticRegression(C=10, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=10000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='liblinear', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=True)

In [59]:
X_test = np.array(X_test)
y_pred = pipe_w2v_lr.predict(X_test.reshape(-1,1))

In [60]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.85      0.82      1652
           1       0.88      0.84      0.86      2315

    accuracy                           0.84      3967
   macro avg       0.84      0.84      0.84      3967
weighted avg       0.85      0.84      0.84      3967



In [61]:
# Best score training
from joblib import dump, load
import time


joblib_file = "C:/Users/kevin/0_projet_satisfaction_client/10_Streamlit/code_pour_streamlit/joblib_file/pipe_w2v_lr.pkl"
dump(pipe_w2v_lr, joblib_file)


['C:/Users/kevin/0_projet_satisfaction_client/10_Streamlit/code_pour_streamlit/joblib_file/pipe_w2v_lr.pkl']

### WORD2VEC UNIQUEMENT pour les modèles REGRESSION LOG / RANDOM FOREST / et Mulitonomial Naïve Bayes

### REGRESSION LOGISTIQUE

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
X = df.drop(['Commentaire', 'Sentiment','preprocessed'], axis=1)
mms = MinMaxScaler()
X = pd.concat([wordvec_df, X], axis=1)
X = pd.DataFrame(mms.fit_transform(X), columns= X.columns)
y = df["Sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state = 0)

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(max_iter= 500)
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
C = [0.01,0.1,1,10,100,120]
grid = dict(solver=solvers,penalty=penalty,C=C)
# define grid search
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=5, scoring='accuracy')
grid_lr = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_lr.best_score_, grid_lr.best_params_))
means = grid_lr.cv_results_['mean_test_score']
stds = grid_lr.cv_results_['std_test_score']
params = grid_lr.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
%%time
print ('Train Accuracy: %.2f'%grid_lr.score(X_train, y_train))
print ('Test Accuracy: %.2f'%grid_lr.score(X_test, y_test))

pred = grid_lr.predict(X_test)

crosstab = pd.crosstab(y_test, pred, rownames=["classe réelle"], colnames=["classe prédite"])
print(crosstab)
plt.figure(figsize=(6,6))
sns.heatmap(crosstab, annot=True, cmap='PuOr');
print(classification_report(y_test, pred))

print(" -- Linear regression optimized --")
pred = grid_lr.predict(X_test)
display(pd.crosstab(y_test, pred,  colnames=["Classe réelle"], rownames=["Classe prédite"]))
print(classification_report(y_test,pred))

In [ ]:
#roc curve
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


pred_probas = grid_lr.predict_proba(X_test)[:,1]


fpr,tpr,_ = roc_curve(y_test, pred_probas)
roc_auc = auc(fpr,tpr)
plt.plot(fpr,tpr,label='area = %.2f' %roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')

plt.show()

### RandomFOREST

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
param_grid = {'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15,100,110,130]}
# define grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring='accuracy')
grid_rf = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_rf.best_score_, grid_rf.best_params_))
means = grid_rf.cv_results_['mean_test_score']
stds = grid_rf.cv_results_['std_test_score']
params = grid_rf.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
%%time
print ('Train Accuracy: %.2f'%grid_lr.score(X_train, y_train))
print ('Test Accuracy: %.2f'%grid_lr.score(X_test, y_test))

pred = grid_rf.predict(X_test)

crosstab = pd.crosstab(y_test, pred, rownames=["classe réelle"], colnames=["classe prédite"])
print(crosstab)
plt.figure(figsize=(6,6))
sns.heatmap(crosstab, annot=True, cmap='PuOr');
print(classification_report(y_test, pred))

print(" -- Linear regression optimized --")
pred = grid_rf.predict(X_test)
display(pd.crosstab(y_test, pred,  colnames=["Classe réelle"], rownames=["Classe prédite"]))
print(classification_report(y_test,pred))

In [ ]:
#roc curve
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


pred_probas = grid_rf.predict_proba(X_test)[:,1]


fpr,tpr,_ = roc_curve(y_test, pred_probas)
roc_auc = auc(fpr,tpr)
plt.plot(fpr,tpr,label='area = %.2f' %roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')

plt.show()

In [ ]:
### Multinomial Naïve Bayes

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
params = {"alpha": np.linspace(1e-10,1,30)}
grid_MN = GridSearchCV(MultinomialNB(), cv= 5, param_grid = params)
grid_MN.fit(X_train, y_train)
print("Meilleur paramètre : ", grid_MN.best_params_, "permettant d'obtenir un score de " , grid_MN.best_score_)
print("Score sur le trainset :",grid_MN.score(X_train,y_train),"; Score sur le testset : ",grid_MN.score(X_test,y_test))
print("On ne fait à priori pas face à un problème d'overfiting")

In [ ]:
y_pred_MNB = grid_MN.predict_proba(X_test)
from sklearn.metrics import auc, roc_curve

fpr , tpr , seuil = roc_curve(y_test, y_pred_MNB[:,1], pos_label= 1)
aucf= auc(fpr, tpr)

plt.plot(fpr, tpr, color='coral', lw=2, label ='auc=%1.5f' % aucf)
plt.plot([0, 1], [0, 1], 'k--')
plt.title("-- Multinomial Naïve Bayes ROC CURVE --")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - specificite (Taux de FN)', fontsize=14)
plt.ylabel('Sensibilite (Taux de VP)', fontsize=14)
plt.legend();
plt.show();


In [ ]:
%%time
print ('Train Accuracy: %.2f'%grid_MN.score(X_train, y_train))
print ('Test Accuracy: %.2f'%grid_MN.score(X_test, y_test))

from sklearn.metrics import classification_report 

pred = grid_MN.predict(X_test)

crosstab = pd.crosstab(y_test, pred, rownames=["classe réelle"], colnames=["classe prédite"])
print(crosstab)
plt.figure(figsize=(6,6))
sns.heatmap(crosstab, annot=True, cmap='PuOr');
print(classification_report(y_test, pred))

print(" -- Multinomial naive baies optimized --")
pred = grid_MN.predict(X_test)
display(pd.crosstab(y_test, pred,  colnames=["Classe réelle"], rownames=["Classe prédite"]))
print(classification_report(y_test,pred))

In [ ]:
words = list(model_w2v.wv.index_to_key)
